In [3]:
import pandas
import numpy

In [4]:
mabc = pandas.read_csv('data/monthly_arrivals_by_country_filtered.csv')

In [5]:
fsi = pandas.read_csv("data/fragilestatesindex_joined.csv")

In [6]:
PSQ_TMS = pandas.read_csv("data/PSQ_TMS.csv", skiprows=range(5), na_values="*")

In [7]:
test = PSQ_TMS[PSQ_TMS["Country Name"] == "Afghanistan"]

In [8]:
test.drop(["Origin / Returned from", "Population type"], axis=1,
    inplace=True)

-c:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
sums = test.iloc[:,1:].sum()

In [10]:
values = pandas.Series(["Afghanistan"], ["Country Name"]).append(sums)

In [11]:
test_df = pandas.DataFrame(values)

In [12]:
test_df.transpose()

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Afghanistan,1.05111e+06,1.2261e+06,3.37648e+06,912292,1.06678e+06,911679,527710,535632,515659,361985,1.31802e+06,1.54837e+06,1.49935e+06,985191


In [13]:
psq_tms_clean = pandas.read_csv("data/PSQ_TMS_filtered.csv")

In [14]:
gdp_info_raw = pandas.read_csv("data/world_bank_gdp/GDP_2011-2015.csv",skiprows=4)

In [15]:
gdp_clean = pandas.read_csv("data/world_bank_gdp/GDP_filtered.csv")

In [16]:
pov_raw = pandas.read_csv("data/world_bank_poverty/Poverty_raw.csv", skiprows=4)

In [17]:
pov_clean = pandas.read_csv("data/world_bank_poverty/Poverty_filtered.csv")

In [18]:
joined_data = pandas.merge(psq_tms_clean[["2013", "Country Name"]], fsi, on="Country Name")

In [19]:
finance_data = pandas.read_csv("data/financial_data.csv")

In [20]:
commodity_data = pandas.read_csv("data/commodities_filtered.csv")

In [21]:
finance_data_filtered = finance_data[["Country", "WEO Subject Code"] +[str(x) for x in range(2000, 2014)]]

In [22]:
finance_data_filtered.replace("n/a", numpy.nan, inplace=True)

-c:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
finance_data_filtered = finance_data_filtered[finance_data["WEO Subject Code"] == "NGDPD"]

In [24]:
finance_data_filtered.drop("WEO Subject Code", axis=1, inplace=True)

In [25]:
finance_data_filtered.rename(columns={str(col): "GDP (Billions of USD), " + str(col) for col in range(2000, 2014)}, inplace=True)

In [26]:
finance_data_filtered.rename(columns={"Country": "Country Name"}, inplace=True)

In [27]:
joined_data = pandas.merge(joined_data, finance_data_filtered, on="Country Name")

In [28]:
joined_data.rename(columns={"2013" : "Total Refugee Count, 2013"}, inplace=True)

In [29]:
joined_data = joined_data.applymap(lambda x: str(x).replace(",",""))

In [30]:
np_joined = joined_data.drop(["Country Name"], axis=1).as_matrix().astype("float64")
np_joined = np_joined[~numpy.isnan(np_joined).any(axis=1)]

In [31]:
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVR

from sklearn import metrics

In [32]:
target = np_joined[:,0]
features = np_joined[:,1:]

ftrain, ftest, ttrain, ttest = train_test_split(features, target, test_size=0.2, random_state=41234)

NameError: name 'normalize' is not defined

In [33]:
linear_svr = LinearSVR(loss='epsilon_insensitive', epsilon=0)

In [4]:
linear_svr.fit(ftrain, ttrain)
preds_svr = linear_svr.predict(ftest)
print numpy.sqrt(metrics.mean_squared_error(ttest, preds_svr))
print metrics.r2_score(ttest, preds_svr)

NameError: name 'linear_svr' is not defined

In [5]:
metadata = pandas.read_csv("data/cpia/CPIA-Series.csv")
metadata[0:5]

,Series Code,Topic,Indicator Name,Long definition,Periodicity,Aggregation method,Limitations and exceptions,Source,Statistical concept and methodology,Development relevance
0,IQ.CPA.HRES.XQ,Public Sector: Policy & institutions,CPIA building human resources rating (1=low to...,Building human resources assesses the national...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...
1,IQ.CPA.BREG.XQ,Public Sector: Policy & institutions,CPIA business regulatory environment rating (1...,Business regulatory environment assesses the e...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...
2,IQ.CPA.DEBT.XQ,Public Sector: Policy & institutions,CPIA debt policy rating (1=low to 6=high),Debt policy assesses whether the debt manageme...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...
3,IQ.CPA.ECON.XQ,Public Sector: Policy & institutions,CPIA economic management cluster average (1=lo...,The economic management cluster includes macro...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...
4,IQ.CPA.REVN.XQ,Public Sector: Policy & institutions,CPIA efficiency of revenue mobilization rating...,Efficiency of revenue mobilization assesses th...,Annual,Unweighted average,The CPIA exercise is intended to capture the q...,"World Bank Group, CPIA database (http://www.wo...",All criteria within each cluster receive equal...,The International Development Association (IDA...


In [6]:
series_codes = metadata["Series Code"]

In [13]:
data = pandas.read_csv("data/cpia/CPIA-Data.csv")
print data.shape
data[numpy.isnan(data["2006"])].shape

(1995, 14)


(150, 14)

In [8]:
import cleanup_cpia as cpia

cpia.cleanup_cpia()

cleanup_cpia.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  axis=1, inplace=True)
/usr/lib/python2.7/site-packages/pandas/core/frame.py:2748: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [16]:
cpia_clean = pandas.read_csv("data/cpia/CPIA-Data_clean.csv")

In [17]:
cpia_clean[0:5]

,Country Name,"IQ.CPA.HRES.XQ, 2005","IQ.CPA.HRES.XQ, 2006","IQ.CPA.HRES.XQ, 2007","IQ.CPA.HRES.XQ, 2008","IQ.CPA.HRES.XQ, 2009","IQ.CPA.HRES.XQ, 2010","IQ.CPA.HRES.XQ, 2011","IQ.CPA.HRES.XQ, 2012","IQ.CPA.HRES.XQ, 2013",...,"IQ.CPA.IRAI.XQ, 2005","IQ.CPA.IRAI.XQ, 2006","IQ.CPA.IRAI.XQ, 2007","IQ.CPA.IRAI.XQ, 2008","IQ.CPA.IRAI.XQ, 2009","IQ.CPA.IRAI.XQ, 2010","IQ.CPA.IRAI.XQ, 2011","IQ.CPA.IRAI.XQ, 2012","IQ.CPA.IRAI.XQ, 2013","IQ.CPA.IRAI.XQ, 2014"
0,Afghanistan,NaN,3.0,3.0,3.0,3.0,3.0,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
1,Afghanistan,NaN,3.0,3.0,3.0,3.0,3.0,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
2,Afghanistan,NaN,3.0,3.0,3.0,3.0,3.0,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
3,Afghanistan,NaN,3.0,3.0,3.0,3.0,3.0,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
4,Afghanistan,NaN,3.0,3.0,3.0,3.0,3.0,3.5,3.5,3.5,...,NaN,2.608333,2.5,2.6,2.75,2.625,2.675,2.675,2.65,2.65
